In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector as conn
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.engine import URL


In [3]:
data = pd.read_csv(r"C:\Users\solat\Downloads\Clean Data_pakwheels.csv\Clean Data_pakwheels.csv")
data


,Unnamed: 0,Company Name,Model Name,Price,Model Year,Location,Mileage,Engine Type,Engine Capacity,Color,Assembly,Body Type,Transmission Type,Registration Status
0,0,Toyota,Vitz,2385000,2017,Islamabad,9869,Petrol,1000,Silver,Imported,Hatchback,Automatic,Un-Registered
1,1,Toyota,Corolla,111000,2019,KPK,11111,Petrol,1300,White,Local,Sedan,Automatic,Registered
2,2,Suzuki,Alto,1530000,2019,KPK,17500,Petrol,660,White,Local,Hatchback,Automatic,Un-Registered
3,3,Suzuki,Alto,1650000,2019,Punjab,9600,Petrol,660,White,Local,Hatchback,Manual,Registered
4,4,Toyota,Corolla,1435000,2010,Islamabad,120000,Petrol,1300,Black,Local,Sedan,Manual,Registered
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46017,46018,Honda,Vezel,3400000,2015,Punjab,32000,Petrol,1500,Black,Imported,Cross Over,Automatic,Un-Registered
46018,46019,Toyota,Aqua,2450000,2015,Punjab,52000,Petrol,1500,Blue,Imported,Hatchback,Automatic,Un-Registered
46019,46020,Honda,Civic,3250000,2015,Punjab,125000,Petrol,1800,Black,Local,Sedan,Automatic,Registered
46020,46021,Toyota,Aqua,3000000,2016,Punjab,60000,Petrol,1500,Black,Imported,Hatchback,Automatic,Registered


In [4]:
data.columns

Index(['Unnamed: 0', 'Company Name', 'Model Name', 'Price', 'Model Year',
       'Location', 'Mileage', 'Engine Type', 'Engine Capacity', 'Color',
       'Assembly', 'Body Type', 'Transmission Type', 'Registration Status'],
      dtype='object')

In [5]:
data.drop("Unnamed: 0",axis=1,inplace=True)

In [6]:
data.head()

,Company Name,Model Name,Price,Model Year,Location,Mileage,Engine Type,Engine Capacity,Color,Assembly,Body Type,Transmission Type,Registration Status
0,Toyota,Vitz,2385000,2017,Islamabad,9869,Petrol,1000,Silver,Imported,Hatchback,Automatic,Un-Registered
1,Toyota,Corolla,111000,2019,KPK,11111,Petrol,1300,White,Local,Sedan,Automatic,Registered
2,Suzuki,Alto,1530000,2019,KPK,17500,Petrol,660,White,Local,Hatchback,Automatic,Un-Registered
3,Suzuki,Alto,1650000,2019,Punjab,9600,Petrol,660,White,Local,Hatchback,Manual,Registered
4,Toyota,Corolla,1435000,2010,Islamabad,120000,Petrol,1300,Black,Local,Sedan,Manual,Registered


In [7]:
data.isna().sum()

Company Name           0
Model Name             0
Price                  0
Model Year             0
Location               0
Mileage                0
Engine Type            0
Engine Capacity        0
Color                  0
Assembly               0
Body Type              0
Transmission Type      0
Registration Status    0
dtype: int64

In [8]:
col_ = []
for col in data.columns:
    cols = col.lower().replace(" ", "_")
    col_.append(cols)
col_

['company_name',
 'model_name',
 'price',
 'model_year',
 'location',
 'mileage',
 'engine_type',
 'engine_capacity',
 'color',
 'assembly',
 'body_type',
 'transmission_type',
 'registration_status']

In [9]:
data.columns = col_

In [12]:
data.columns

Index(['company_name', 'model_name', 'price', 'model_year', 'location',
       'mileage', 'engine_type', 'engine_capacity', 'color', 'assembly',
       'body_type', 'transmission_type', 'registration_status'],
      dtype='object')

In [24]:
cat_cols = data.dtypes[data.dtypes == "O"].index
cat_cols

Index(['company_name', 'model_name', 'location', 'engine_type', 'color',
       'assembly', 'body_type', 'transmission_type', 'registration_status'],
      dtype='object')

In [27]:
num_cols = data.dtypes[data.dtypes != "O"].index
num_cols

Index(['price', 'model_year', 'mileage', 'engine_capacity'], dtype='object')

In [31]:

pd.api.types.is_string_dtype(data["company_name"])

True

In [34]:
# we have two data types in dataset, thus we can write if else to create sql_table
def get_sql_table():

    final_col = []
    for key, val in data.items():
        if pd.api.types.is_string_dtype(val):
            cat_col_name = key + " varchar(50)"
            final_col.append(cat_col_name)
        else:
            num_col_name = key + " int"
            final_col.append(num_col_name)

    return final_col

get_sql_table()


['company_name varchar(50)',
 'model_name varchar(50)',
 'price int',
 'model_year int',
 'location varchar(50)',
 'mileage int',
 'engine_type varchar(50)',
 'engine_capacity int',
 'color varchar(50)',
 'assembly varchar(50)',
 'body_type varchar(50)',
 'transmission_type varchar(50)',
 'registration_status varchar(50)']

In [ ]:
mydb = conn.connect(
    host = "localhost",
    user = "root",
    password = "password"
)

cur = mydb.cursor()
cur.execute("show databases")
cur.fetchall()

In [64]:
list_of_col = get_sql_table()
list_of_col

['company_name varchar(50)',
 'model_name varchar(50)',
 'price int',
 'model_year int',
 'location varchar(50)',
 'mileage int',
 'engine_type varchar(50)',
 'engine_capacity int',
 'color varchar(50)',
 'assembly varchar(50)',
 'body_type varchar(50)',
 'transmission_type varchar(50)',
 'registration_status varchar(50)']

In [89]:
comma_col = [list_of_col[i] + "," for i in range(len(list_of_col)-1)]
comma_col.append(list_of_col[-1])
comma_col

['company_name varchar(50),',
 'model_name varchar(50),',
 'price int,',
 'model_year int,',
 'location varchar(50),',
 'mileage int,',
 'engine_type varchar(50),',
 'engine_capacity int,',
 'color varchar(50),',
 'assembly varchar(50),',
 'body_type varchar(50),',
 'transmission_type varchar(50),',
 'registration_status varchar(50)']

In [91]:
single_quote = ' '.join(comma_col)
table_data = f"({single_quote})"
cr_table = 'create table if not exists car_data'
table_tmp = cr_table + " " + table_data
table_tmp
# table_data

'create table if not exists car_data (company_name varchar(50), model_name varchar(50), price int, model_year int, location varchar(50), mileage int, engine_type varchar(50), engine_capacity int, color varchar(50), assembly varchar(50), body_type varchar(50), transmission_type varchar(50), registration_status varchar(50))'

In [93]:
cur.execute("use car")

table = table_tmp
cur.execute(table)
cur.execute("select * from car_data")
cur.fetchall()


[]

In [101]:
url_object = URL.create(
    "mysql",
    username="root",
    password="password",
    database="car"
)

engine = create_engine(url_object)
data.to_sql('car_data', engine,if_exists="append",index=False)

46022

In [ ]:
cur.execute('select * from car_data')
# print(cur.fetchall())